In [8]:
import pandas as pd
import FinanceDataReader as fdr
from time import time
from concurrent.futures import ProcessPoolExecutor

df_krx = fdr.StockListing('KRX')
df_krx['SymbolName'] = df_krx['Symbol'] + df_krx['Name']
codes = df_krx['SymbolName']

def getPrice(code):
    df_price = fdr.DataReader(code[:6], '2018-12-01', '2019-12-22') # 주가 가져오기
    df_price = df_price[['Close']]
    df_price.columns = [code[6:]]
    return df_price

if __name__ == '__main__':
    start = time()
    pool = ProcessPoolExecutor(max_workers=10)
    results = list(pool.map(getPrice,codes))
    stocks = pd.concat(results, axis=1)
    end = time()
    print(end-start)
    stocks.to_excel('datas.xlsx')

c:\users\administrator\appdata\local\programs\python\python37-32\lib\site-packages\FinanceDataReader\krx\listing.py:40: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_finder = json_normalize(jo, 'block1')


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [4]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

BASE_URL='https://finance.naver.com/sise/sise_market_sum.nhn?sosok='

KOSPI_CODE = 0
KOSDAK_CODE = 1
START_PAGE = 1
fields = []

def main(code):
    # total_page을 가져오기 위한 requests
    res = requests.get(BASE_URL + str(code) + "&page=" + str(START_PAGE))
    page_soup = BeautifulSoup(res.text, 'lxml')

    # total_page 가져오기
    total_page_num = page_soup.select_one('td.pgRR > a')
    total_page_num = int(total_page_num.get('href').split('=')[-1])

    #가져올 수 있는 항목명들을 추출
    ipt_html = page_soup.select_one('div.subcnt_sise_item_top')
    global fields
    fields = [item.get('value') for item in ipt_html.select('input')]

    # page마다 정보를 긁어오게끔 하여 result에 저장
    result = [crawl(code,str(page)) for page in range(1,total_page_num+1)]

    # page마다 가져온 정보를 df에 하나로 합침
    df = pd.concat(result, axis=0,ignore_index=True)

    # 엑셀로 내보내기
    df.to_excel('NaverFinance.xlsx')

def crawl(code, page):
    global fields
    data = {'menu': 'market_sum',
            'fieldIds':  fields,
            'returnUrl': BASE_URL + str(code) + "&page=" + str(page)}
    # requests.get 요청대신 post 요청
    res = requests.post('https://finance.naver.com/sise/field_submit.nhn', data=data)

    page_soup = BeautifulSoup(res.text, 'lxml')
    # 크롤링할 table html 가져오기
    table_html = page_soup.select_one('div.box_type_l')

    # Column명
    header_data  = [item.get_text().strip() for item in table_html.select('thead th')][1:-1]

    # 종목명 + 수치 추출 (a.title = 종목명, td.number = 기타 수치)
    inner_data = [item.get_text().strip() for item in table_html.find_all(lambda x:
                                                                           (x.name == 'a' and
                                                                            'tltle' in x.get('class', [])) or
                                                                           (x.name == 'td' and
                                                                            'number' in x.get('class', []))
                                                                           )]

    # page마다 있는 종목의 순번 가져오기
    no_data = [item.get_text().strip() for item in table_html.select('td.no')]
    number_data = np.array(inner_data)

    # 가로 x 세로 크기에 맞게 행렬화
    number_data.resize(len(no_data), len(header_data ))

    # 한 페이지에서 얻은 정보를 모아 DataFrame로 만들어 리턴
    df = pd.DataFrame(data=number_data, columns=header_data )
    return df


main(KOSPI_CODE)

  ERROR: Could not find a version that satisfies the requirement FinanceDataReader (from versions: none)
ERROR: No matching distribution found for FinanceDataReader
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
#준비물1 가격정보 datas.xlsx
#준비물2 재무정보 NaverFinance.xlsx
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

df_finance = pd.read_excel('NaverFinance.xlsx') # NaverFinance - 재무정보
df_price = pd.read_excel('datas.xlsx',index_col=0) # datas - 가격정보 with 날짜정보 그대로

MONTH_AGO = datetime(2019,12,20)+relativedelta(months=-1) # 1달전 주가구하기, 그냥 오늘 날로 하려면 datetime.today() + relativedelta(months=-1)
MONTH_AGO = MONTH_AGO.strftime('%Y-%m-%d')

YEAR_AGO =  datetime(2019,12,20)+relativedelta(years=-1) # 1년전 주가구하기, 그냥 오늘 날로 하려면 datetime.today()+relativedelta(years=-1)
YEAR_AGO = YEAR_AGO.strftime('%Y-%m-%d')

price_month_ago =[]
price_year_ago =[]

for index, row in df_finance.iterrows():
    name = row['종목명']
    if name in df_price.columns:
        price_month_ago.append(df_price.loc[MONTH_AGO, name] ) # 준비물인 datas에서 1달 전 주가 구하기
        price_year_ago.append(df_price.loc[YEAR_AGO, name]) # 준비물인 datas에서 1년 전 주가 구하기
    else :
        # datas에서 종목이 존재하지 않는 경우(우선주, ETF 등은 없음)가 있으므로 제외
        price_month_ago.append(0) 
        price_year_ago.append(0)        

df_finance['price_month_ago'] = price_month_ago # 1달 전 주가를 구해서 새로운 COLUMN으로 추가
df_finance['price_year_ago'] = price_year_ago # 1년 전 주가를 구해서 새로운 COLUMN으로 추가

df_finance =df_finance[df_finance['price_month_ago']!= 0] # 기준의 가격이 0이 아닌 종목만(우선주 등 없는 데이터 제외)

df_finance = df_finance.reset_index(drop=True)
df_finance = df_finance.loc[:200] # 시총 상위 200개만 추출, df_finance의 원천인 NaverFinance.xlsx은 원래 시총순 정렬이므로 순서대로 200개 자름

df_finance['BPR'] = 1/df_finance['PBR'].astype(float) # BPR = 1/PBR 
df_finance['1/PER'] = 1/df_finance['PER'].str.replace(',', '').astype(float) # Per가 1000이 넘어 1,000인 형태가 존재하므로 replace 수행 후 type 변경
df_finance['RANK_BPR']  = df_finance['BPR'].rank(method='max', ascending=False) # BPR의 순위 
df_finance['RANK_1/PER']  = df_finance['1/PER'].rank(method='max', ascending=False) # 1/PER의 순위
df_finance['RANK_VALUE']  = (df_finance['RANK_BPR'] + df_finance['RANK_1/PER'])/ 2 # 순위의 평균을 구함 > 가치평가 순위 

df_finance = df_finance.sort_values(by=['RANK_VALUE']) # 가치평가 순위로 정렬 
df_finance = df_finance.reset_index(drop=True)
df_finance = df_finance.loc[:75] # 가치평가로 상위 75개만 추출
# ----- 1차 가치평가 종료 -----

df_finance['현재가'] = df_finance['현재가'].str.replace(',', '').astype(float)

#1달 등락률 계산
df_finance['momentum_month'] = df_finance['현재가'] - df_finance['price_month_ago'] #오늘주가 - 1달 전 주가
df_finance['1달 등락률'] = (df_finance['현재가'] - df_finance['price_month_ago']) /  df_finance['현재가']

#1년 등락률 계산
df_finance['momentum_year'] = df_finance['현재가'] - df_finance['price_year_ago'] #오늘주가 - 1년 전 주가
df_finance['1년 등락률'] = (df_finance['현재가'] - df_finance['price_year_ago']) /  df_finance['현재가']

df_finance['FINAL_MOMENTUM'] = df_finance['1년 등락률'] - df_finance['1달 등락률'] # 1년 등락률 - 1달 등락률
df_finance['RANK_MOMENTUM'] = df_finance['FINAL_MOMENTUM'].rank(method='max', ascending=False) # 모멘텀의 순위 
# ----- 2차 모멘텀평가 종료 -----

df_finance['FINAL_RANK'] = (df_finance['RANK_VALUE']  + df_finance['RANK_MOMENTUM'])/2 # 가치 순위와 모멘텀 순위의 합산
df_finance = df_finance.sort_values(by=['FINAL_RANK'], ascending=[True])
df_finance = df_finance.reset_index(drop=True)

df_finance.to_excel('momentum+value.xlsx') # 최종 선정된 주식들 목록 
print(df_finance.head())

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

  Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'


You should consider upgrading via the 'python -m pip install --upgrade pip' command.
